In [1]:
from langgraph_sdk import get_client
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)
await client.assistants.search()

[{'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21',
  'graph_id': 'task_maistro',
  'created_at': '2025-01-15T23:49:59.417849+00:00',
  'updated_at': '2025-01-15T23:49:59.417849+00:00',
  'config': {},
  'metadata': {'created_by': 'system'},
  'version': 1,
  'name': 'task_maistro'}]

In [7]:
import httpx
from langchain_core.messages import convert_to_messages
from langchain_core.messages import HumanMessage

In [8]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Send Erik his t-shirt gift this weekend."
user_input_2 = "Get cash and pay nanny for 2 weeks. Do this by Friday."
config = {"configurable": {"user_id": "Lance"}}
graph_name = "task_maistro" 

first_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="enqueue",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Send Erik his t-shirt gift this weekend.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_cyIALnlw5qwyVoQLMiMFRron)
 Call ID: call_cyIALnlw5qwyVoQLMiMFRron
  Args:
    update_type: todo
================================= Tool Message =================================

Document 8ee305ba-fea0-43ef-93fb-2e71b806d48a updated:
Plan: Update the deadline for sending Erik his t-shirt gift to this weekend, which is January 19, 2025.
Added content: 2025-01-19T23:59:00

Document 8ee305ba-fea0-43ef-93fb-2e71b806d48a updated:
Plan: Update the status of the task to 'in progress' to reflect the current state of the task.
Added content: in progress
================================== Ai Message ==================================

I've updated the deadline for sending Erik his t-shirt gift to this weekend. Let me know if there's anything else you need!
=======